随着引擎和 SQL 执行的停止，我们准备开始一些 Alchemy。 SQLAlchemy Core 和 ORM 的核心元素是 SQL 表达式语言，它允许流畅、可组合地构建 SQL 查询。这些查询的基础是表示表和列等数据库概念的 Python 对象。这些对象统称为数据库元数据。

SQLAlchemy 中数据库元数据最常见的基础对象称为MetaData 、 Table和Column 。下面的部分将说明如何在面向 Core 的风格和面向 ORM 的风格中使用这些对象。

[文档链接]{https://docs.sqlalchemy.org/en/20/tutorial/metadata.html}

In [7]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///database.sqlite", echo=True)

## 使用Table对象设置元数据

In [8]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [9]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [10]:
# 查看name字段
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [12]:
# 查看表所有字段
user_table.c.keys()

['id', 'name', 'fullname']

In [13]:
# 查看表主键
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [14]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

当在一个对象中使用ForeignKey对象时 Column定义，我们可以省略其数据类型 Column ;它是从相关列的数据类型自动推断出来的，在上面的示例中是user_account.id列的Integer数据类型。

In [15]:
metadata_obj.create_all(engine)

2024-12-24 22:56:26,968 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-24 22:56:26,969 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-24 22:56:26,970 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 22:56:26,971 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-24 22:56:26,971 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 22:56:26,972 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-12-24 22:56:26,972 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 22:56:26,973 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-12-24 22:56:26,973 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 22:56:26,974 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-12-24 22:56:26,975 INFO sqlalchemy.engine.Engine [no key 0.00050s] ()
2024-12-24 22:56:26,980 INFO sqlalchemy.engine.Engine 
C

## 使用ORM声明式定义表元数据

In [17]:
# 建立声明性基础
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [18]:
Base.metadata

MetaData()

In [20]:
Base.registry

In [22]:
# 声明映射类
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

In [23]:
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [24]:
class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [25]:
Base.metadata.create_all(engine)

2024-12-24 23:06:53,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-24 23:06:53,477 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-24 23:06:53,477 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:06:53,478 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-24 23:06:53,480 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:06:53,480 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-12-24 23:06:53,481 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:06:53,482 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-12-24 23:06:53,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:06:53,483 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-12-24 23:06:53,483 INFO sqlalchemy.engine.Engine [no key 0.00030s] ()
2024-12-24 23:06:53,487 INFO sqlalchemy.engine.

## 表反射

In [26]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2024-12-24 23:08:29,420 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-24 23:08:29,420 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2024-12-24 23:08:29,421 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:08:29,422 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-12-24 23:08:29,422 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2024-12-24 23:08:29,424 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2024-12-24 23:08:29,424 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:08:29,425 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2024-12-24 23:08:29,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-24 23:08:29,425 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [27]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)